In [1]:
import os

print(os.getcwd())

q:\AD project\rag


In [2]:
os.chdir("..")

In [3]:
print(os.getcwd())

q:\AD project


In [4]:
from src.book_preprocess import get_book_data

In [21]:
book_doc_data = get_book_data(100)

100%|██████████| 10/10 [00:00<00:00, 12.53it/s]


In [23]:
len(book_doc_data[12]['text'].split(" "))

105

In [24]:
book_doc_data[13]['text']

'Having established the hurdle rate, we turn our attention to measuring the returns on an investment.\nIn chapter 5, we evaluate three alternative ways of measuring returns - conventional accounting earnings, cash flows and time-weighted cash flows (where we consider both how large the cash flows are and when they are anticipated to come in).\nIn chapter 6, we consider some of the potential side-costs which might not be captured in any of these measures, including costs that may be created for existing investments by taking a new investment, and side-benefits, such as options to enter new markets and to expand product lines that may be embedded in new investments, and synergies, especially when the new investment is the acquisition of another firm.\n\n'

In [25]:
book_doc_data[14]['text']

'Every business, no matter how large and complex it is, is ultimately funded with a mix of borrowed money (debt) and owner’s funds (equity).\nWith a publicly trade firm, debt may take the form of bonds and equity is usually common stock.\nIn a private business, debt is more likely to be bank loans and an owner’s savings represent equity.\nWhile we consider the existing mix of debt and equity and its implications for the minimum acceptable hurdle rate as part of the investment principle, we throw open the question of whether the existing mix is the right one in the financing principle section.\nWhile there might be regulatory and other real world constraints on the financing mix that a business can use, there is ample room for flexibility within these constraints.\nWe begin this section in chapter 7, by looking at the range of choices that exist for both private businesses and publicly traded firms between debt and equity.\nWe then turn to the\n\n'

In [26]:
book_doc_data[15]['text']

'question of whether the existing mix of financing used by a business is the “optimal” one, given our objective function of maximizing firm value, in chapter 8.\nWhile the tradeoff between the benefits and costs of borrowing are established in qualitative terms first, we also look at two quantitative approaches to arriving at the optimal mix in chapter 8.\nIn the first approach, we examine the specific conditions under which the optimal financing mix is the one that minimizes the minimum acceptable hurdle rate.\nIn the second approach, we look at the effects on firm value of changing the financing mix.\n\n'

In [27]:
book_doc_data[-1]

{'text': 'Risk management as a discipline, has evolved unevenly across different functional areas.\nIn finance, the preoccupation has been with the effect of risk on discount rates and little attention has been paid to the potential upside of risk until recently; real options represent the first real attempt to bring in the potential profits of being exposed to risk.\nIn strategy, risk management has been a side story to the main focus on competitive advantages and barriers to entry.\nIn practice, risk management at most organizations is splintered, with little communication between those who assess risk and those who make decisions based upon those risk assessments.This book is an attempt to bridge the chasm not only between different functional areas = finance, strategy and operations – but also between different parts of organizations where the responsibility for risk management lie today.\nIn the process, it makes the argument that good risk management lies at the heart of successf

In [28]:
len(book_doc_data)

3881

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

book_langchain_doc = []
curr_page = 0
curr_book = ""

for bd in book_doc_data:
    page_num = bd['page_num_coordinates'][0][0]['page_num']
    
    if curr_book!= bd['book_source']:
        curr_book = bd['book_source']
        curr_page = 0
    
    if page_num!= curr_page:
        curr_page = page_num
        book_langchain_doc.append(
            Document(
                page_content=bd['text'],
                metadata={
                    'page_coordinates': bd['coordinates'],
                }
            )
        )
    elif page_num == curr_page:

In [19]:
page_num

2

In [20]:
bd

{'text': 'Let me begin this preface with a confession of a few of my own biases.\nFirst, I believe that theory, and the models that flow from it, should provide us with the tools to understand, analyze and solve problems.\nThe test of a model or theory then should not be based upon its elegance but upon its usefulness in problem solving.\nSecond, there is little in corporate financial theory, in my view, that is new and revolutionary.\nThe core principles of corporate finance are common sense ones, and have changed little over time.\nThat should not be surprising.\nCorporate finance is only a few decades old and people have been running businesses for thousands of years, and it would be exceedingly presumptuous of us to believe that they were in the dark until corporate finance theorists came along and told them what to do.\nTo be fair, it is true that corporate financial theory has made advances in taking common sense principles and providing them with structure, but these advances ha